In [32]:
using CSV, DataFrames, Random, Gurobi

In [33]:
# Read in the data
df = CSV.read("../../data/preprocessed/stock_prices_train.csv",DataFrame)
# remove missing 
df = df[completecases(df),:]
# drop RowID, Date, SecuritiesCode from df
y = df[:, :Target]
#X = df[:, Not([1,2,3,12])] why removing 3 and 12?
X = df[:, Not([1,2])]

466507-element Vector{Float64}:
  0.0076267384477344
 -0.0160284951024042
  0.0167420814479637
 -0.0111259457053849
  0.0022502250225022
  0.0035922766052987
  0.0062639821029082
 -0.0022232103156958
  0.0164884135472371
  0.0258658483121438
  0.0192307692307691
 -0.0054507337526205
 -0.0126475548060708
  ⋮
  0.0
 -0.0055865921787709
 -0.0126404494382021
  0.0042674253200569
 -0.0254957507082153
 -0.0290697674418605
 -0.0014970059880239
  0.0269865067466266
 -0.0014598540145985
  0.0175438596491228
  0.014367816091954
  0.0184135977337109

In [34]:
first(df, 10)

Row,Date,SecuritiesCode,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Date,Int64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2017-01-04,1301,2734.0,2755.0,2730.0,31400,0.00073046,2742.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-05,1301,2743.0,2747.0,2735.0,17900,0.00291971,2738.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-06,1301,2734.0,2744.0,2720.0,19900,-0.0010917,2740.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00401141
4,2017-01-10,1301,2745.0,2754.0,2735.0,24200,-0.00510018,2748.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00801989
5,2017-01-11,1301,2748.0,2752.0,2737.0,9300,-0.0032955,2745.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0062152
6,2017-01-12,1301,2745.0,2747.0,2703.0,28700,-0.00661278,2731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00953249
7,2017-01-13,1301,2707.0,2730.0,2707.0,19400,-0.0066568,2722.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00957651
8,2017-01-16,1301,2725.0,2725.0,2696.0,20100,0.00297841,2704.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2017-01-17,1301,2702.0,2704.0,2682.0,18400,0.00185598,2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00112243


In [35]:
first(X_train, 10)

Row,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2734.0,2755.0,2730.0,31400,0.00073046,2742.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2743.0,2747.0,2735.0,17900,0.00291971,2738.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2734.0,2744.0,2720.0,19900,-0.0010917,2740.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00401141
4,2745.0,2754.0,2735.0,24200,-0.00510018,2748.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00801989
5,2748.0,2752.0,2737.0,9300,-0.0032955,2745.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0062152
6,2745.0,2747.0,2703.0,28700,-0.00661278,2731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00953249
7,2707.0,2730.0,2707.0,19400,-0.0066568,2722.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00957651
8,2725.0,2725.0,2696.0,20100,0.00297841,2704.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2702.0,2704.0,2682.0,18400,0.00185598,2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00112243


In [36]:
df_test = CSV.read("../../data/preprocessed/stock_prices_supplemental.csv",DataFrame)
first(df_test, 10)

Row,Date,SecuritiesCode,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Date,Int64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2021-12-06,1301,2982.0,2982.0,2965.0,8900,-0.00326264,2971.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-12-07,1301,2998.0,3065.0,2990.0,19100,0.00981997,3065.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-12-08,1301,3080.0,3080.0,3035.0,11600,0.00648298,3055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00333699
4,2021-12-09,1301,3050.0,3085.0,3025.0,11700,-0.00644122,3085.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0162612
5,2021-12-10,1301,3100.0,3105.0,3050.0,14700,-0.00810373,3105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0179237
6,2021-12-13,1301,3110.0,3110.0,3075.0,12100,0.00980392,3085.0,0.0,0.0,0.0,0.0,0.0,0.0,1.60457e-5
7,2021-12-14,1301,3085.0,3085.0,3055.0,10200,0.0113269,3060.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2021-12-15,1301,3060.0,3095.0,3060.0,8500,-0.0096,3090.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0209269
9,2021-12-16,1301,3100.0,3135.0,3085.0,16900,-0.0242326,3125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0355595


In [37]:
df = vcat(df,df_test)
first(df, 10)

Row,Date,SecuritiesCode,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Date,Int64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2017-01-04,1301,2734.0,2755.0,2730.0,31400,0.00073046,2742.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-05,1301,2743.0,2747.0,2735.0,17900,0.00291971,2738.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-06,1301,2734.0,2744.0,2720.0,19900,-0.0010917,2740.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00401141
4,2017-01-10,1301,2745.0,2754.0,2735.0,24200,-0.00510018,2748.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00801989
5,2017-01-11,1301,2748.0,2752.0,2737.0,9300,-0.0032955,2745.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0062152
6,2017-01-12,1301,2745.0,2747.0,2703.0,28700,-0.00661278,2731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00953249
7,2017-01-13,1301,2707.0,2730.0,2707.0,19400,-0.0066568,2722.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00957651
8,2017-01-16,1301,2725.0,2725.0,2696.0,20100,0.00297841,2704.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2017-01-17,1301,2702.0,2704.0,2682.0,18400,0.00185598,2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00112243


In [5]:
df[!,:Drawdown] = zeros(size(df,1))
security_code = df[1204,:SecuritiesCode]
df_prev = df[1:1203,:]
idx = df_prev[!, :SecuritiesCode] .== security_code
print("a")
df[1204,:Drawdown] = maximum(df_prev[idx, :Target]) - df[1204,:Target]

a

0.0349326393141456

In [ ]:
df[!,:Drawdown] = zeros(size(df,1))

for i in 2:size(df,1)
    security_code = df[i,:SecuritiesCode]
    df_prev = df[1:i-1,:]
    idx = df_prev[!, :SecuritiesCode] .== security_code
    df[i,:Drawdown] = maximum(df_prev[idx, :Target], init=0) - df[i,:Target]
end
first(df, 10)

In [114]:
using Dates, Gurobi, JuMP

function prescription_baseline(df, C, D, lambda)
    Random.seed!(42)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)

    #extract the part of the dataset that correspond to day 2022/06/23, which is the day before the test day
    idx = df[!,:Date] .== Date(2022,6,23)
    df_2022_06_23 = df[idx,:]
    
    # extract the 200 performing stocks based on the value of Target for the date "2022-06-23"
    df_2022_06_23_top = sort(df_2022_06_23, :Target, rev=true)[1:200,:]
    # same for the 200 worst performing stocks
    df_2022_06_23_bottom = sort(df_2022_06_23, :Target, rev=false)[1:200,:]

    #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
    @variable(m, w[1:400]>=0);
    @objective(m, Max, sum(w[i] * df_2022_06_23_top[!,"Target"][i] for i in 1:200) - sum(w[200+i] * df_2022_06_23_bottom[!,"Target"][i] for i in 1:200) 
                + lambda*(C - (sum(w[i] * df_2022_06_23_top[!,"Drawdown"][i] for i in 1:200) + sum(w[200+i] * df_2022_06_23_bottom[!,"Drawdown"][i] for i in 1:200))/600))
    
    #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
    #we are relaxing it as it depends on R
#     @constraint(m, (sum(w[i] * df_2022_06_23_top[!,"Drawdown"][i] for i in 1:200) + sum(w[i] * df_2022_06_23_bottom[!,"Drawdown"][i] for i in 1:200))/600 <= C)
    
    #sum of w should be equal to D
    @constraint(m, sum(w) == D)
    @constraint(m, [i in 1:400], w[i]<= 10)
    optimize!(m)
    #return the value of w
    return value.(w)

end

prescription_baseline (generic function with 2 methods)

In [115]:
w_opt_baseline = prescription_baseline(df, 0.01, 600,1000)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


400-element Vector{Float64}:
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
  0.0
 10.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [45]:
idx = df[!,:Date] .== Date(2022,6,24)
df_2022_06_24 = df[idx,:]
    
# extract the 200 performing stocks based on the value of Target for the date "2022-06-23"
df_2022_06_24_top = sort(df_2022_06_24, :Target, rev=true)[1:200,:]
# same for the 200 worst performing stocks
df_2022_06_24_bottom = sort(df_2022_06_24, :Target, rev=false)[1:200,:]

Row,Date,SecuritiesCode,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Date,Int64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2022-06-24,4772,185.0,221.0,181.0,6564100,-0.068,203.0,157.6,0.0565635,149.775,0.0498302,149.717,0.0431327,0.299527
2,2022-06-24,3660,232.0,239.0,229.0,1076100,-0.0472103,235.0,229.85,0.045342,203.025,0.0442826,194.3,0.0397477,0.176675
3,2022-06-24,3031,1840.0,1899.0,1816.0,1153500,-0.0436836,1826.0,1533.25,0.0506802,1487.47,0.0425432,1408.03,0.0375405,0.160047
4,2022-06-24,1407,4155.0,4460.0,3805.0,1700800,-0.0431566,4390.0,4272.0,0.0362882,4516.0,0.037786,4693.58,0.0370481,0.232346
5,2022-06-24,4058,1323.0,1388.0,1317.0,19400,-0.0393701,1387.0,1367.75,0.0228054,1412.65,0.0299982,1476.85,0.0287428,0.154196
6,2022-06-24,6094,1842.0,1887.0,1781.0,83800,-0.0387513,1876.0,1757.75,0.0395632,1809.33,0.0426152,1877.73,0.039437,0.156398
7,2022-06-24,4449,1138.0,1249.0,1138.0,2151600,-0.0384924,1233.0,1104.95,0.0573985,1079.35,0.0669403,1130.9,0.0615741,0.220974
8,2022-06-24,7630,4945.0,4975.0,4930.0,27600,-0.0365112,4975.0,4897.0,0.00878699,4886.5,0.0091148,4816.75,0.00886153,0.0606075
9,2022-06-24,4054,1050.0,1073.0,1038.0,56600,-0.0362745,1042.0,1180.25,0.0270262,1202.78,0.0501727,1199.32,0.0483266,0.208072


In [116]:
sum(w_opt_baseline[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) - sum(w_opt_baseline[200+i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200)

31.227483585453953

In [117]:
function oracle_policy(df_2022_06_24_top, df_2022_06_24_bottom, C, D, lambda)
    Random.seed!(42)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)


    #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
    @variable(m, w[1:400]>=0);
    @objective(m, Max, sum(w[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) - sum(w[200+i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200) 
    +lambda*(C - (sum(w[i] * df_2022_06_24_top[!,"Drawdown"][i] for i in 1:200) + sum(w[200+i] * df_2022_06_24_bottom[!,"Drawdown"][i] for i in 1:200))/600))
    
    #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C (we are relaxing it)
    #@constraint(m, (sum(w[i] * df_2022_06_24_top[!,"Drawdown"][i] for i in 1:200) + sum(w[i] * df_2022_06_24_bottom[!,"Drawdown"][i] for i in 1:200))/600 <= C)
    
    #sum of w should be equal to D
    @constraint(m, sum(w) == D)
    @constraint(m, [i in 1:400], w[i]<= 10)
    optimize!(m)
    #return the value of w
    return value.(w)

end

oracle_policy (generic function with 2 methods)

In [118]:
w_opt_oracle = oracle_policy(df_2022_06_24_top, df_2022_06_24_bottom, 0.01, 600, 100)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


400-element Vector{Float64}:
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [120]:
sum(w_opt_oracle[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) - sum(w_opt_oracle[200+i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200)

37.02153322403812

In [77]:
using NearestNeighbors, Statistics
df_train = df[df[:, :Date] .!= Date(2022,6,24),:]
df_test = df[df[:, :Date] .== Date(2022,6,24),:]
X_train = df_train[:, Not([1,2,7,15])]
y_train = df_train[:, :Target]

X_test = df_test[:, Not([1,2,7,15])]

num_neighbors = 25
kdtree = KDTree(Matrix(X_train)'; leafsize = 10)
indices_test, dists_test = knn(kdtree, Matrix(X_test)', num_neighbors)

y_pred = zeros(2000)
for i in 1:2000
    y_pred[i] = mean(y_train[indices_test[i,:]])
end
#compute for each stock the mean drawdown for all its neighbors
drawdown_pred = zeros(200)
for i in 1:2000
    drawdown_pred[i] = mean(df_train[indices_test[i,:],:Drawdown])

end
#we select the top 200 stocks to buy and the bottom 200 stocks to shorts based on the predictions
indeces_top = sortperm(y_pred, rev=true)[1:200]
indeces_bottom = sortperm(y_pred, rev=false)[1:200]

y_pred_top = y_pred[indeces_top]
y_pred_bottom = y_pred[indeces_bottom]

drawdown_pred_top = drawdown_pred[indeces_top]
drawdown_pred_bottom = drawdown_pred[indeces_bottom]

[1693083, 2026018, 352985, 1949937, 1907759, 1360911, 1985741, 1922401, 443258, 605920, 565624, 1509372, 1922625, 353424, 1922538, 1334464, 1907195, 516023, 581672, 386075, 1971262, 207014, 937716, 538104, 1971314]


In [78]:
size(drawdown_top)

(200,)

In [121]:
using NearestNeighbors
function knn_prescription(y_pred_top, y_pred_bottom, drawdown_pred_top, drawdown_pred_bottom, C, D, lambda)
    Random.seed!(42)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0) 
    
    #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
    @variable(m, w[1:400]>=0);
    @objective(m, Max, sum(w[i] * y_pred_top[i] for i in 1:200) - sum(w[200+i] * y_pred_bottom[i] for i in 1:200)
    + lambda * (C - (sum(w[i] * drawdown_pred_top[i] for i in 1:200) + sum(w[200+i] * drawdown_pred_bottom[i] for i in 1:200))/600))
    
    #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
    #@constraint(m, (sum(w[i] * drawdown_top[i] for i in 1:200) + sum(w[i] * drawdown_bottom[i] for i in 1:200))/600 <= C)
    
    #sum of w should be equal to D
    @constraint(m, sum(w) == D)
    @constraint(m, [i in 1:400], w[i]<= 10)
    optimize!(m)
    #return the value of w
    return value.(w)

end

knn_prescription (generic function with 2 methods)

In [122]:
w_opt_knn = knn_prescription(y_pred_top, y_pred_bottom, drawdown_pred_top, drawdown_pred_bottom, 0.05, 600, 100)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


400-element Vector{Float64}:
  0.0
 10.0
  0.0
 10.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 10.0
 10.0
  0.0
  ⋮
  0.0
  0.0
 10.0
 10.0
 10.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [107]:
sum(w_opt_knn)

600.0

In [123]:
sum(w_opt_knn[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) - sum(w_opt_knn[200+i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200)

18.65946756222579

In [110]:
sum(w_opt_knn[i] for i in 1:200)

0.0

In [167]:
function saa_prescription(df_train, C, D, lambda)
    Random.seed!(42)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)
    
    y_train = df_train[:, :Target]
    drawdown_train = df_train[:, :Drawdown]
    #for each stock in df_2022_06_24_top and df_2022_06_24_bottom, compute the mean of the historical returns over the training set
    y_pred = zeros(2000)
    unique_securities = unique(df_train[:, :SecuritiesCode])
    for i in 1:2000
        y_pred[i] = mean(y_train[df_train[:, :SecuritiesCode] .== unique_securities[i]])
    end
    drawdown_pred = zeros(2000)
    for i in 1:2000
        drawdown_pred[i] = mean(drawdown_train[df_train[:, :SecuritiesCode] .== unique_securities[i]])
    end
    
    indeces_top = sortperm(y_pred, rev=true)[1:200]
    indeces_bottom = sortperm(y_pred, rev=false)[1:200]
    
    y_pred_top = y_pred[indeces_top]
    y_pred_bottom = y_pred[indeces_bottom]

    drawdown_pred_top = drawdown_pred[indeces_top]
    drawdown_pred_bottom = drawdown_pred[indeces_bottom]
    
    @variable(m, w[1:400]>=0);
    @objective(m, Max, sum(w[i] * y_pred_top[i] for i in 1:200) - sum(w[200+i] * y_pred_bottom[i] for i in 1:200) 
    +lambda*(C - (sum(w[i] * drawdown_pred_top[i] for i in 1:200) + sum(w[200+i] * drawdown_pred_bottom[i] for i in 1:200))/600))
    
    #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C (we are relaxing it)
    #@constraint(m, (sum(w[i] * df_2022_06_24_top[!,"Drawdown"][i] for i in 1:200) + sum(w[i] * df_2022_06_24_bottom[!,"Drawdown"][i] for i in 1:200))/600 <= C)
    
    #sum of w should be equal to D
    @constraint(m, sum(w) == D)
    @constraint(m, [i in 1:400], w[i]<= 10)
    optimize!(m)
    #return the value of w
    return value.(w)

end

saa_prescription (generic function with 1 method)

In [168]:
w_opt_saa = saa_prescription(df_train, df_2022_06_24_top, df_2022_06_24_bottom, 0.05, 600, 100)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


400-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
 10.0
 10.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [169]:
sum(w_opt_saa[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) - sum(w_opt_saa[200+i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200)

7.9625212937519105

In [ ]:
function OCT_prescription(y_pred_top, y_pred_bottom, drawdown_pred_top, drawdown_pred_bottom, C, D, lambda)
    Random.seed!(42)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0) 
    
    #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
    @variable(m, w[1:400]>=0);
    @objective(m, Max, sum(w[i] * y_pred_top[i] for i in 1:200) - sum(w[200+i] * y_pred_bottom[i] for i in 1:200)
    + lambda * (C - (sum(w[i] * drawdown_pred_top[i] for i in 1:200) + sum(w[200+i] * drawdown_pred_bottom[i] for i in 1:200))/600))
    
    #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
    #@constraint(m, (sum(w[i] * drawdown_top[i] for i in 1:200) + sum(w[i] * drawdown_bottom[i] for i in 1:200))/600 <= C)
    
    #sum of w should be equal to D
    @constraint(m, sum(w) == D)
    @constraint(m, [i in 1:400], w[i]<= 10)
    optimize!(m)
    #return the value of w
    return value.(w)

end

In [170]:
lnr = IAI.OptimalTreeRegressor(random_seed=15095, minbucket=30, max_depth = 4, cp = 0.001)
  
IAI.fit!(lnr, X_train, y_train);
@show pred = IAI.predict(lnr, X_test)
leafs_train =IAI.apply(lnr, X_train)
leafs_test = IAI.apply(lnr, X_test)

y_pred = zeros(2000)
for i in 1:2000
    y_pred[i] = mean(y_train[leafs_train.==leafs_test[i]])
end

#compute for each stock the mean drawdown for all its neighbors
drawdown_pred = zeros(2000)
for i in 1:2000
    drawdown_pred[i] = mean(df_train[leafs_train.==leafs_test[i],:Drawdown])

end
#we select the top 200 stocks to buy and the bottom 200 stocks to shorts based on the predictions
indeces_top = sortperm(y_pred, rev=true)[1:200]
indeces_bottom = sortperm(y_pred, rev=false)[1:200]

y_pred_top = y_pred[indeces_top]
y_pred_bottom = y_pred[indeces_bottom]

drawdown_pred_top = drawdown_pred[indeces_top]
drawdown_pred_bottom = drawdown_pred[indeces_bottom]

w_opt_OCT = OCT_prescription(y_pred_top, y_pred_bottom, drawdown_pred_top, drawdown_pred_bottom, 0.05, 600, 100)

┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ nothing nothing:nothing
┌ Warning: Interpretable AI license expires soon: 2022-12-31T00:00:00. If you need to renew, please send us the following machine ID:
│ 90feab63202e1ce4738c9f8316af63ad83f6344d3b63e878e7f54e8248f156e6
└ @ nothing nothing:nothing
Excessive output truncated after 10486169 bytes.

pred = IAI.predict(lnr, X_test) = 

200-element Vector{Float64}:
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 ⋮
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228
 0.1084811155037228

In [173]:
first(X_test, 10)

Row,Open,High,Low,Volume,Target,AdjustedClose,return_20days,volatility_20days,return_40days,volatility_40days,return_60days,volatility_60days,Drawdown
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2250.0,2253.0,2229.0,29200,0.00762674,2239.0,2231.65,0.00892715,2266.25,0.0116943,2285.6,0.0118727,0.0844581
2,2236.0,2250.0,2219.0,45700,-0.0160285,2229.0,2232.5,0.00893857,2264.2,0.0114043,2282.85,0.0118722,0.108113
3,2234.0,2249.0,2222.0,42800,0.0167421,2246.0,2233.3,0.00891622,2261.47,0.011037,2281.2,0.0116445,0.0753428
4,2255.0,2264.0,2207.0,108700,-0.0111259,2210.0,2232.2,0.00964752,2257.82,0.0112837,2279.68,0.011576,0.103211
5,2223.0,2248.0,2217.0,56000,0.00225023,2247.0,2233.35,0.0103385,2254.82,0.011607,2277.9,0.0113902,0.0898346
6,2247.0,2261.0,2218.0,61100,0.00359228,2222.0,2233.55,0.010647,2252.57,0.0111572,2276.7,0.0109947,0.0884926
7,2232.0,2233.0,2213.0,48500,0.00626398,2227.0,2233.85,0.0106545,2251.38,0.010893,2275.6,0.0110005,0.0858209
8,2227.0,2235.0,2213.0,60400,-0.00222321,2235.0,2234.95,0.0106429,2251.22,0.0106518,2274.93,0.0109695,0.0943081
9,2238.0,2253.0,2228.0,75700,0.0164884,2249.0,2235.65,0.0105037,2250.38,0.0102704,2274.67,0.010988,0.0755964


In [ ]:
sum(w_opt_OCT[i] * df_2022_06_24_top[!,"Target"][i] for i in 1:200) - sum(w_opt_OCT[200+i] * df_2022_06_24_bottom[!,"Target"][i] for i in 1:200)

In [ ]:
dates = unique(df_train[:, :Date])

In [151]:
function prescriptive_coef_knn(df_train, C, D, lambda, dates)
    N = length(dates)
    df_train = df_train[df_train[:, :Date] .∈ dates, :]
    X_train = df_train[:, Not([1,2,7,15])]
    y_train = df_train[:, :Target]
    num_neighbors = 25
    kdtree = KDTree(Matrix(X_train)'; leafsize = 10)
    
    cost_inner = []
    cost_knn = []
    m_glob = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m_glob, "OutputFlag", 0)
    @variable(m_glob, t[1:N]>=0);
    @variable(m_glob, w_glob[1:400]>=0, base_name="w_glob");
    @objective(m_glob, Min, sum(t[i] for i in 1:N))
    j=0
    
    #loop over all the dates of the dataset df_train
    for date in dates
        #extract the part of the dataset that correspond to the date date
        idx = df_train[!,:Date] .== date
        df_test = df_train[idx,:]
        X_test = df_test[:, Not([1,2,7,15])]
        y_test = df_test[:, :Target]
        indices_test, dists_test = knn(kdtree, Matrix(X_test)', num_neighbors)
        y_pred = zeros(2000)
        for i in 1:2000
            y_pred[i] = mean(y_train[indices_test[i,:]])
        end
        #compute for each stock the mean drawdown for all its neighbors
        drawdown_pred = zeros(200)
        for i in 1:2000
            drawdown_pred[i] = mean(df_train[indices_test[i,:],:Drawdown])

        end
        #we select the top 200 stocks to buy and the bottom 200 stocks to shorts based on the predictions
        indeces_top = sortperm(y_pred, rev=true)[1:200]
        indeces_bottom = sortperm(y_pred, rev=false)[1:200]

        y_pred_top = y_pred[indeces_top]
        y_pred_bottom = y_pred[indeces_bottom]

        drawdown_pred_top = drawdown_pred[indeces_top]
        drawdown_pred_bottom = drawdown_pred[indeces_bottom]
        
        y_test_top = y_test[indeces_top]
        y_test_bottom = y_test[indeces_bottom]
        
        Random.seed!(42)
        m = Model(Gurobi.Optimizer)
        set_optimizer_attribute(m, "OutputFlag", 0) 
        
        #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
        @variable(m, w[1:400]>=0);
        @objective(m, Max, sum(w[i] * y_pred_top[i] for i in 1:200) - sum(w[200+i] * y_pred_bottom[i] for i in 1:200)
        + lambda * (C - (sum(w[i] * drawdown_pred_top[i] for i in 1:200) + sum(w[200+i] * drawdown_pred_bottom[i] for i in 1:200))/600))
        
        #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
        #@constraint(m, (sum(w[i] * drawdown_top[i] for i in 1:200) + sum(w[i] * drawdown_bottom[i] for i in 1:200))/600 <= C)
        
        #sum of w should be equal to D
        @constraint(m, sum(w) == D)
        @constraint(m, [i in 1:400], w[i]<= 10)
        optimize!(m)
        
        w_opt = value.(w)
        
        push!(cost_knn, sum(w_opt[i] * y_test_top[i] for i in 1:200) - sum(w_opt[200+i] * y_test_bottom[i] for i in 1:200))

        j+=1
        
        @constraint(m_glob, sum(w_glob[i] * y_test_top[i] for i in 1:200) - sum(w_glob[200+i] * y_test_bottom[i] for i in 1:200) <= t[j])
        
        m_inner = Model(Gurobi.Optimizer)
        set_optimizer_attribute(m_inner, "OutputFlag", 0) 
        @variable(m_inner, w_inner[1:400]>=0, base_name="w_inner");
        @objective(m_inner, Min, sum(w_inner[i] * y_test_top[i] for i in 1:200) - sum(w_inner[200+i] * y_test_bottom[i] for i in 1:200) )
        @constraint(m_inner, sum(w_inner) == D)
        @constraint(m_inner, [i in 1:400], w_inner[i]<= 10)
        optimize!(m_inner)
        push!(cost_inner, objective_value(m_inner))
        
    end
    @constraint(m_glob, sum(w_glob) == D)
    @constraint(m_glob, [i in 1:400], w_glob[i]<= 10)
    optimize!(m_glob)
    cost_glob = objective_value(m_glob)

    prescriptive_coef = (cost_glob - sum(cost_knn))/(cost_glob - sum(cost_inner))

    return prescriptive_coef
end

prescriptive_coef_knn (generic function with 1 method)

In [152]:
prescription_coef = prescriptive_coef(df_train,  0.05, 600, 100, dates)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expi

cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - ex

cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - ex

cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - ex

cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - ex

...

abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - e

cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - ex

abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - e

cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
abSet parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
cSet parameter Username
Academic license - for non-commercial use only - ex

0.5413535870706153

In [159]:
function prescriptive_coef_baseline(df_train, C, D, lambda, dates)
    N = length(dates)
    df_train = df_train[df_train[:, :Date] .∈ dates, :]
    cost_inner = []
    cost_baseline = []
    m_glob = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m_glob, "OutputFlag", 0)
    @variable(m_glob, t[1:N]>=0);
    @variable(m_glob, w_glob[1:400]>=0, base_name="w_glob");
    @objective(m_glob, Min, sum(t[i] for i in 1:N))
    j=0
    
    #loop over all the dates of the dataset df_train
    for i in 2:length(unique(df_train[:, :Date]))
        #extract the part of the dataset that correspond to the date date
        print(unique(df_train[:, :Date])[i])
        idx_test = df_train[!,:Date] .== unique(df_train[:, :Date])[i]
        idx_train = df_train[!,:Date] .== unique(df_train[:, :Date])[i-1]
        df_test = df_train[idx_test,:]
        train = df_train[idx_train,:]
        #extract the 200 performing stocks based on the value of Target for the date date
        indeces_top = sortperm(train, rev=true)[1:200]
        indeces_bottom = sortperm(train, rev=false)[1:200]
        
        
        df_test_date_top = df_test[indeces_top]
        df_test_date_bottom = df_test[indeces_bottom]
        
        y_test_top = df_test_date_top[:, :Target]
        y_test_bottom = df_test_date_bottom[:, :Target]
        
        df_train_date_top = train[indeces_top]
        df_train_date_bottom = train[indeces_bottom]
        
        
        y_train_top = df_train_date_top[:, :Target]
        y_train_bottom = df_train_date_bottom[:, :Target]
        
        drawdown_train_top = df_train_date_top[:, :Drawdown]
        drawdown_train_bottom = df_train_date_bottom[:, :Drawdown]

        Random.seed!(42)
        m = Model(Gurobi.Optimizer)
        set_optimizer_attribute(m, "OutputFlag", 0) 
        
        #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
        @variable(m, w[1:400]>=0);
        @objective(m, Max, sum(w[i] * y_train_top[i] for i in 1:200) - sum(w[200+i] * y_train_bottom[i] for i in 1:200)
        + lambda * (C - (sum(w[i] * drawdown_train_top[i] for i in 1:200) + sum(w[200+i] * drawdown_train_bottom[i] for i in 1:200))/600))
        
        #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
        #@constraint(m, (sum(w[i] * drawdown_top[i] for i in 1:200) + sum(w[i] * drawdown_bottom[i] for i in 1:200))/600 <= C)
        
        #sum of w should be equal to D
        @constraint(m, sum(w) == D)
        @constraint(m, [i in 1:400], w[i]<= 10)
        optimize!(m)
        
        w_opt = value.(w)
        
        push!(cost_baseline, sum(w_opt[i] * y_test_top[i] for i in 1:200) - sum(w_opt[200+i] * y_test_bottom[i] for i in 1:200))

        j+=1
        
        @constraint(m_glob, sum(w_glob[i] * y_test_top[i] for i in 1:200) - sum(w_glob[200+i] * y_test_bottom[i] for i in 1:200) <= t[j])
        
        m_inner = Model(Gurobi.Optimizer)
        set_optimizer_attribute(m_inner, "OutputFlag", 0) 
        @variable(m_inner, w_inner[1:400]>=0, base_name="w_inner");
        @objective(m_inner, Min, sum(w_inner[i] * y_test_top[i] for i in 1:200) - sum(w_inner[200+i] * y_test_bottom[i] for i in 1:200) )
        @constraint(m_inner, sum(w_inner) == D)
        @constraint(m_inner, [i in 1:400], w_inner[i]<= 10)
        optimize!(m_inner)
        push!(cost_inner, objective_value(m_inner))
        
    end
    @constraint(m_glob, sum(w_glob) == D)
    @constraint(m_glob, [i in 1:400], w_glob[i]<= 10)
    optimize!(m_glob)
    cost_glob = objective_value(m_glob)

    prescriptive_coef = (cost_glob - sum(cost_baseline))/(cost_glob - sum(cost_inner))

    return prescriptive_coef
end

prescriptive_coef_baseline (generic function with 1 method)

In [160]:
prescription_coef_baseline = prescriptive_coef_baseline(df_train,  0.05, 600, 100, dates)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-01-05Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-01-06Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-01-10Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-01-11Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-01-12Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license 

2017-03-09Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-03-10Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-03-13Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-03-14Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-03-15Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-03-16Set parameter Username
Academi

2017-05-15Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-05-16Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-05-17Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-05-18Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-05-19Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-05-22Set parameter Username
Academi

2017-07-13Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-07-14Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-07-18Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-07-19Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-07-20Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-07-21Set parameter Username
Academi

2017-09-14Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-09-15Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-09-19Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-09-20Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-09-21Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2017-09-22Set parameter Username
Academi

...

2021-10-20Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-10-21Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-10-22Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-10-25Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-10-26Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-10-27Set parameter Username
Academi

2021-12-22Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-12-23Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-12-24Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-12-27Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-12-28Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2021-12-29Set parameter Username
Academi

2022-03-01Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-03-02Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-03-03Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-03-04Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-03-07Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-03-08Set parameter Username
Academi

2022-05-09Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-05-10Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-05-11Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-05-12Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-05-13Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
2022-05-16Set parameter Username
Academi

-5.847704642623147

In [ ]:
function prescriptive_coef_saa(df_train, C, D, lambda, dates)
    N = length(dates)
    df_train = df_train[df_train[:, :Date] .∈ dates, :]
    
    X_train = df_train[:, Not([1,2,7,15])]
    y_train = df_train[:, :Target]
    
    drawdown_train = df_train[:, :Drawdown]
    #for each stock in df_2022_06_24_top and df_2022_06_24_bottom, compute the mean of the historical returns over the training set
    y_pred = zeros(2000)
    unique_securities = unique(df_train[:, :SecuritiesCode])
    for i in 1:2000
        y_pred[i] = mean(y_train[df_train[:, :SecuritiesCode] .== unique_securities[i]])
    end
    drawdown_pred = zeros(2000)
    for i in 1:2000
        drawdown_pred[i] = mean(drawdown_train[df_train[:, :SecuritiesCode] .== unique_securities[i]])
    end
    
    indeces_top = sortperm(y_pred, rev=true)[1:200]
    indeces_bottom = sortperm(y_pred, rev=false)[1:200]
    y_pred_top = y_pred[indeces_top]
    y_pred_bottom = y_pred[indeces_bottom]

    drawdown_pred_top = drawdown_pred[indeces_top]
    drawdown_pred_bottom = drawdown_pred[indeces_bottom]
    
    cost_inner = []
    cost_saa = []
    m_glob = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m_glob, "OutputFlag", 0)
    @variable(m_glob, t[1:N]>=0);
    @variable(m_glob, w_glob[1:400]>=0, base_name="w_glob");
    @objective(m_glob, Min, sum(t[i] for i in 1:N))
    j=0
    
    #loop over all the dates of the dataset df_train
    for date in dates
        #extract the part of the dataset that correspond to the date date
        idx = df_train[!,:Date] .== date
        df_test = df_train[idx,:]
        X_test = df_test[:, Not([1,2,7,15])]
        y_test = df_test[:, :Target]

        
        y_test_top = y_test[indeces_top]
        y_test_bottom = y_test[indeces_bottom]
        
        Random.seed!(42)
        m = Model(Gurobi.Optimizer)
        set_optimizer_attribute(m, "OutputFlag", 0) 
        
        #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
        @variable(m, w[1:400]>=0);
        @objective(m, Max, sum(w[i] * y_pred_top[i] for i in 1:200) - sum(w[200+i] * y_pred_bottom[i] for i in 1:200)
        + lambda * (C - (sum(w[i] * drawdown_pred_top[i] for i in 1:200) + sum(w[200+i] * drawdown_pred_bottom[i] for i in 1:200))/600))
        
        #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
        #@constraint(m, (sum(w[i] * drawdown_top[i] for i in 1:200) + sum(w[i] * drawdown_bottom[i] for i in 1:200))/600 <= C)
        
        #sum of w should be equal to D
        @constraint(m, sum(w) == D)
        @constraint(m, [i in 1:400], w[i]<= 10)
        optimize!(m)
        
        w_opt = value.(w)
        
        push!(cost_saa, sum(w_opt[i] * y_test_top[i] for i in 1:200) - sum(w_opt[200+i] * y_test_bottom[i] for i in 1:200))

        j+=1
        
        @constraint(m_glob, sum(w_glob[i] * y_test_top[i] for i in 1:200) - sum(w_glob[200+i] * y_test_bottom[i] for i in 1:200) <= t[j])
        
        m_inner = Model(Gurobi.Optimizer)
        set_optimizer_attribute(m_inner, "OutputFlag", 0) 
        @variable(m_inner, w_inner[1:400]>=0, base_name="w_inner");
        @objective(m_inner, Min, sum(w_inner[i] * y_test_top[i] for i in 1:200) - sum(w_inner[200+i] * y_test_bottom[i] for i in 1:200) )
        @constraint(m_inner, sum(w_inner) == D)
        @constraint(m_inner, [i in 1:400], w_inner[i]<= 10)
        optimize!(m_inner)
        push!(cost_inner, objective_value(m_inner))
        
    end
    @constraint(m_glob, sum(w_glob) == D)
    @constraint(m_glob, [i in 1:400], w_glob[i]<= 10)
    optimize!(m_glob)
    cost_glob = objective_value(m_glob)

    prescriptive_coef = (cost_glob - sum(cost_saa))/(cost_glob - sum(cost_inner))

    return prescriptive_coef
end

In [ ]:
prescription_coef_saa = prescriptive_coef_saa(df_train,  0.05, 600, 100, dates)

In [ ]:
function prescriptive_coef_OCT(lnr, df_train, C, D, lambda, dates)
    N = length(dates)
    df_train = df_train[df_train[:, :Date] .∈ dates, :]
    X_train = df_train[:, Not([1,2,7,15])]
    y_train = df_train[:, :Target]
    leafs_train =IAI.apply(lnr, X_train)
    
    cost_inner = []
    cost_OCT = []
    m_glob = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m_glob, "OutputFlag", 0)
    @variable(m_glob, t[1:N]>=0);
    @variable(m_glob, w_glob[1:400]>=0, base_name="w_glob");
    @objective(m_glob, Min, sum(t[i] for i in 1:N))
    j=0
    
    #loop over all the dates of the dataset df_train
    for date in dates
        #extract the part of the dataset that correspond to the date date
        idx = df_train[!,:Date] .== date
        df_test = df_train[idx,:]
        X_test = df_test[:, Not([1,2,7,15])]
        y_test = df_test[:, :Target]
        
        leafs_test = IAI.apply(lnr, X_test)

        y_pred = zeros(2000)
        for i in 1:2000
            y_pred[i] = mean(y_train[leafs_train.==leafs_test[i]])
        end

        #compute for each stock the mean drawdown for all its neighbors
        drawdown_pred = zeros(2000)
        for i in 1:2000
            drawdown_pred[i] = mean(df_train[leafs_train.==leafs_test[i],:Drawdown])

        end
        #we select the top 200 stocks to buy and the bottom 200 stocks to shorts based on the predictions
        indeces_top = sortperm(y_pred, rev=true)[1:200]
        indeces_bottom = sortperm(y_pred, rev=false)[1:200]

        y_pred_top = y_pred[indeces_top]
        y_pred_bottom = y_pred[indeces_bottom]

        drawdown_pred_top = drawdown_pred[indeces_top]
        drawdown_pred_bottom = drawdown_pred[indeces_bottom]

        
        y_test_top = y_test[indeces_top]
        y_test_bottom = y_test[indeces_bottom]
        
        Random.seed!(42)
        m = Model(Gurobi.Optimizer)
        set_optimizer_attribute(m, "OutputFlag", 0) 
        
        #we select the top 200 stocks to buy and the bottom 200 stocks to shorts
        @variable(m, w[1:400]>=0);
        @objective(m, Max, sum(w[i] * y_pred_top[i] for i in 1:200) - sum(w[200+i] * y_pred_bottom[i] for i in 1:200)
        + lambda * (C - (sum(w[i] * drawdown_pred_top[i] for i in 1:200) + sum(w[200+i] * drawdown_pred_bottom[i] for i in 1:200))/600))
        
        #add the constraint that the max of drawdowns on the day 2022/06/23 weighted by w is less that C
        #@constraint(m, (sum(w[i] * drawdown_top[i] for i in 1:200) + sum(w[i] * drawdown_bottom[i] for i in 1:200))/600 <= C)
        
        #sum of w should be equal to D
        @constraint(m, sum(w) == D)
        @constraint(m, [i in 1:400], w[i]<= 10)
        optimize!(m)
        
        w_opt = value.(w)
        
        push!(cost_knn, sum(w_opt[i] * y_test_top[i] for i in 1:200) - sum(w_opt[200+i] * y_test_bottom[i] for i in 1:200))

        j+=1
        
        @constraint(m_glob, sum(w_glob[i] * y_test_top[i] for i in 1:200) - sum(w_glob[200+i] * y_test_bottom[i] for i in 1:200) <= t[j])
        
        m_inner = Model(Gurobi.Optimizer)
        set_optimizer_attribute(m_inner, "OutputFlag", 0) 
        @variable(m_inner, w_inner[1:400]>=0, base_name="w_inner");
        @objective(m_inner, Min, sum(w_inner[i] * y_test_top[i] for i in 1:200) - sum(w_inner[200+i] * y_test_bottom[i] for i in 1:200) )
        @constraint(m_inner, sum(w_inner) == D)
        @constraint(m_inner, [i in 1:400], w_inner[i]<= 10)
        optimize!(m_inner)
        push!(cost_inner, objective_value(m_inner))
        
    end
    @constraint(m_glob, sum(w_glob) == D)
    @constraint(m_glob, [i in 1:400], w_glob[i]<= 10)
    optimize!(m_glob)
    cost_glob = objective_value(m_glob)

    prescriptive_coef = (cost_glob - sum(cost_knn))/(cost_glob - sum(cost_inner))

    return prescriptive_coef
end

In [ ]:
prescription_coef_OCT = prescriptive_coef_OCT(lnr, df_train,  0.05, 600, 100, dates)

In [ ]:
df_prescriptiveness = DataFrame()
dates = unique(df_train[!,:Date])
for prop in [1, 0.8, 0.6, 0.4, 0.2, 0.1]
    #take the last prop of the dates
    dates = dates[length(dates) - round(Int, length(dates)*prop):end]
    #compute the coefficient of prescriptiveness for each method
    coef_knn = prescriptive_coef_knn(df_train, C, D, lambda, dates)
    coef_OCT = prescriptive_coef_OCT(lnr, df_train, C, D, lambda, dates)
    coef_baseline = prescriptive_coef_baseline(df_train, C, D, lambda, dates)
    coef_saa = prescriptive_coef_saa(df_train, C, D, lambda, dates)
    #save coefs in dataframe with the number of datapoints = length(dates)*2000
    push!(df_prescriptiveness, (nb_data_points = length(dates)*2000, knn = coef_knn, OCT = coef_OCT, baseline = coef_baseline, saa = coef_saa))
end
#plot on the same graph the coefficient of prescriptiveness for each method
plot(df_prescriptiveness[!,:nb_data_points], df_prescriptiveness[!,:knn], label="knn", xlabel="number of datapoints", ylabel="coefficient of prescriptiveness", title="coefficient of prescriptiveness as a function of the number of datapoints")
plot!(df_prescriptiveness[!,:nb_data_points], df_prescriptiveness[!,:OCT], label="OCT")
plot!(df_prescriptiveness[!,:nb_data_points], df_prescriptiveness[!,:baseline], label="baseline")
plot!(df_prescriptiveness[!,:nb_data_points], df_prescriptiveness[!,:saa], label="saa")
